In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
print(pd.__version__)

In [ ]:
base_sample = pd.read_csv('/kaggle/input/samples-bt/sample_10.csv')
base_sample

## Load Data and Get Probabilities from Targets

We are getting probabilities to train a cross-encoder that compares two passages and decides the long-run probability of a reviewer rating one passage as easier than the other.

In [ ]:
train_path = "../input/commonlitreadabilityprize/train.csv"
df = pd.read_csv(train_path)
df.head(n=3)

### Create all unique pairs of excerpts in the training set

Should be $2 |A|\cdot|B| - 2|A|$ total samples in the training set.

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error
from itertools import combinations, permutations

def cartesian(df_1, df_2):
    df_1['key'] = 0
    df_2['key'] = 0

    df_cartesian = df_1.merge(df_2, how='outer', on='key')
    return df_cartesian

def create_training_set(df_train):
    combination_idx = list(combinations(range(len(df_train)), 2))
    print(f'number of combination indicies: {len(combination_idx)}')

    left_idx, right_idx = zip(*combination_idx)
    left, right = df_train.loc[left_idx, :], df_train.loc[right_idx, :]
    train = pd.concat([left.reset_index(), right.reset_index()], axis=1)
    train.columns = ['index', 'id', 'url_legal', 'license', 'excerpt_l', 'target_l',
           'standard_error_l', 'index', 'id', 'url_legal', 'license', 'excerpt_r',
           'target_r', 'standard_error_r']

    assert len(train) == len(combination_idx)
    return train


def create_samples_train(raw_df, samples):
    df = raw_df[~raw_df.id.isin(samples.id)]
    print(f'number of df rows without samples: {len(df)}')
    kfold = KFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True)
#     skfold = StratifiedKFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True) # TODO: stratify by standard error?
    splits= kfold.split(df)
    for i,(train_index, test_index) in enumerate(splits):
        print(train_index.shape,test_index.shape)
        df_train, df_test = df.iloc[train_index,:].reset_index(drop=True), df.iloc[test_index,:].reset_index(drop=True)
        break
    df_train = pd.concat([df_train, samples])
    print(f'total shape: {len(df_train)}')
    ab = cartesian(samples, df_train)
    ba = cartesian(df_train, samples)
    print(f'|A||B| = {len(ab)}')
    print(f'|B||A| = {len(ba)}')
    return pd.concat([ab, ba]), df_test

In [ ]:
train_batch_size = 16
train_size = 100000
RANDOM_STATE = 42

In [ ]:
kfold = KFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True)
#     skfold = StratifiedKFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True) # TODO: stratify by standard error?
splits= kfold.split(df)
for i,(train_index, test_index) in enumerate(splits):
    print(train_index.shape,test_index.shape)
    raw = df.iloc[train_index,:].reset_index(drop=True)
    break

In [ ]:
df_train, df_test = create_samples_train(df, base_sample)

In [ ]:
df_train = df_train[df_train.id_x != df_train.id_y]

In [ ]:
len(df_train), 2269*10*2 - 2*10

In [ ]:
!pip install sentence_transformers

## Cross Encoder Class

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
import logging
import os
from typing import Dict, Type, Callable, List
import transformers
import torch
from torch import nn
from torch.optim import Optimizer
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm, trange
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.evaluation import SentenceEvaluator

logger = logging.getLogger(__name__)

class CrossEncoder():
    def __init__(self, model_name:str, num_labels:int = None, max_length:int = None, device:str = None, tokenizer_args:Dict = {},
                 default_activation_function = None, kl = False):
        """
        A CrossEncoder takes exactly two sentences / texts as input and either predicts
        a score or label for this sentence pair. It can for example predict the similarity of the sentence pair
        on a scale of 0 ... 1.
        It does not yield a sentence embedding and does not work for individually sentences.
        :param model_name: Any model name from Huggingface Models Repository that can be loaded with AutoModel. We provide several pre-trained CrossEncoder models that can be used for common tasks
        :param num_labels: Number of labels of the classifier. If 1, the CrossEncoder is a regression model that outputs a continous score 0...1. If > 1, it output several scores that can be soft-maxed to get probability scores for the different classes.
        :param max_length: Max length for input sequences. Longer sequences will be truncated. If None, max length of the model will be used
        :param device: Device that should be used for the model. If None, it will use CUDA if available.
        :param tokenizer_args: Arguments passed to AutoTokenizer
        :param default_activation_function: Callable (like nn.Sigmoid) about the default activation function that should be used on-top of model.predict(). If None. nn.Sigmoid() will be used if num_labels=1, else nn.Identity()
        """
        self.kl = kl
        self.config = AutoConfig.from_pretrained(model_name)
        classifier_trained = True
        if self.config.architectures is not None:
            classifier_trained = any([arch.endswith('ForSequenceClassification') for arch in self.config.architectures])

        if num_labels is None and not classifier_trained:
            num_labels = 1

        if num_labels is not None:
            self.config.num_labels = num_labels

        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, config=self.config)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, **tokenizer_args)
        self.max_length = max_length

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
            logger.info("Use pytorch device: {}".format(device))

        self._target_device = torch.device(device)

        if default_activation_function is not None:
            self.default_activation_function = default_activation_function
            try:
                self.config.sbert_ce_default_activation_function = util.fullname(self.default_activation_function)
            except Exception as e:
                logger.warning("Was not able to update config about the default_activation_function: {}".format(str(e)) )
        elif hasattr(self.config, 'sbert_ce_default_activation_function') and self.config.sbert_ce_default_activation_function is not None:
            self.default_activation_function = util.import_from_string(self.config.sbert_ce_default_activation_function)()
        else:
            self.default_activation_function = nn.Sigmoid() if self.config.num_labels == 1 else nn.Identity()

    def smart_batching_collate(self, batch):
        texts = [[] for _ in range(len(batch[0].texts))]
        labels = []

        for example in batch:
            for idx, text in enumerate(example.texts):
                texts[idx].append(text.strip())

            labels.append(example.label)

        tokenized = self.tokenizer(*texts, padding=True, truncation='longest_first', return_tensors="pt", max_length=self.max_length)
        labels = torch.tensor(labels, dtype=torch.float if (self.config.num_labels == 1 or self.kl) else torch.long).to(self._target_device)

        for name in tokenized:
            tokenized[name] = tokenized[name].to(self._target_device)

        return tokenized, labels

    def smart_batching_collate_text_only(self, batch):
        texts = [[] for _ in range(len(batch[0]))]

        for example in batch:
            for idx, text in enumerate(example):
                texts[idx].append(text.strip())

        tokenized = self.tokenizer(*texts, padding=True, truncation='longest_first', return_tensors="pt", max_length=self.max_length)

        for name in tokenized:
            tokenized[name] = tokenized[name].to(self._target_device)

        return tokenized

    def fit(self,
            train_dataloader: DataLoader,
            evaluator: SentenceEvaluator = None,
            epochs: int = 1,
            loss_fct = None,
            activation_fct = nn.Identity(),
            scheduler: str = 'WarmupLinear',
            warmup_steps: int = 10000,
            optimizer_class: Type[Optimizer] = transformers.AdamW,
            optimizer_params: Dict[str, object] = {'lr': 2e-5},
            weight_decay: float = 0.01,
            evaluation_steps: int = 0,
            output_path: str = None,
            save_best_model: bool = True,
            max_grad_norm: float = 1,
            use_amp: bool = False,
            callback: Callable[[float, int, int], None] = None,
            ):
        """
        Train the model with the given training objective
        Each training objective is sampled in turn for one batch.
        We sample only as many batches from each objective as there are in the smallest one
        to make sure of equal training with each dataset.
        :param train_dataloader: DataLoader with training InputExamples
        :param evaluator: An evaluator (sentence_transformers.evaluation) evaluates the model performance during training on held-out dev data. It is used to determine the best model that is saved to disc.
        :param epochs: Number of epochs for training
        :param loss_fct: Which loss function to use for training. If None, will use nn.BCEWithLogitsLoss() if self.config.num_labels == 1 else nn.CrossEntropyLoss()
        :param activation_fct: Activation function applied on top of logits output of model.
        :param scheduler: Learning rate scheduler. Available schedulers: constantlr, warmupconstant, warmuplinear, warmupcosine, warmupcosinewithhardrestarts
        :param warmup_steps: Behavior depends on the scheduler. For WarmupLinear (default), the learning rate is increased from o up to the maximal learning rate. After these many training steps, the learning rate is decreased linearly back to zero.
        :param optimizer_class: Optimizer
        :param optimizer_params: Optimizer parameters
        :param weight_decay: Weight decay for model parameters
        :param evaluation_steps: If > 0, evaluate the model using evaluator after each number of training steps
        :param output_path: Storage path for the model and evaluation files
        :param save_best_model: If true, the best model (according to evaluator) is stored at output_path
        :param max_grad_norm: Used for gradient normalization.
        :param use_amp: Use Automatic Mixed Precision (AMP). Only for Pytorch >= 1.6.0
        :param callback: Callback function that is invoked after each evaluation.
                It must accept the following three parameters in this order:
                `score`, `epoch`, `steps`
        """
        train_dataloader.collate_fn = self.smart_batching_collate

        if use_amp:
            from torch.cuda.amp import autocast
            scaler = torch.cuda.amp.GradScaler()

        self.model.to(self._target_device)

        if output_path is not None:
            os.makedirs(output_path, exist_ok=True)

        self.best_score = -9999999
        num_train_steps = int(len(train_dataloader) * epochs)

        # Prepare optimizers
        param_optimizer = list(self.model.named_parameters())

        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

        optimizer = optimizer_class(optimizer_grouped_parameters, **optimizer_params)

        if isinstance(scheduler, str):
            scheduler = SentenceTransformer._get_scheduler(optimizer, scheduler=scheduler, warmup_steps=warmup_steps, t_total=num_train_steps)

        if loss_fct is None:
            loss_fct = nn.BCEWithLogitsLoss() if self.config.num_labels == 1 else nn.CrossEntropyLoss()


        skip_scheduler = False
        for epoch in trange(epochs, desc="Epoch"):
            training_steps = 0
            self.model.zero_grad()
            self.model.train()

            for features, labels in tqdm(train_dataloader, desc="Iteration", smoothing=0.05):
                if use_amp:
                    with autocast():
                        model_predictions = self.model(**features, return_dict=True)
                        logits = activation_fct(model_predictions.logits)
                        if self.config.num_labels == 1:
                            logits = logits.view(-1)
                        loss_value = loss_fct(logits, labels)

                    scale_before_step = scaler.get_scale()
                    scaler.scale(loss_value).backward()
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_grad_norm)
                    scaler.step(optimizer)
                    scaler.update()

                    skip_scheduler = scaler.get_scale() != scale_before_step
                else:
                    model_predictions = self.model(**features, return_dict=True)
                    logits = activation_fct(model_predictions.logits)
                    if self.config.num_labels == 1:
                        logits = logits.view(-1)
                    loss_value = loss_fct(logits, labels)
                    loss_value.backward()
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_grad_norm)
                    optimizer.step()

                optimizer.zero_grad()

                if not skip_scheduler:
                    scheduler.step()

                training_steps += 1

                if evaluator is not None and evaluation_steps > 0 and training_steps % evaluation_steps == 0:
                    self._eval_during_training(evaluator, output_path, save_best_model, epoch, training_steps, callback)

                    self.model.zero_grad()
                    self.model.train()

            if evaluator is not None:
                self._eval_during_training(evaluator, output_path, save_best_model, epoch, -1, callback)



    def predict(self, sentences: List[List[str]],
               batch_size: int = 32,
               show_progress_bar: bool = None,
               num_workers: int = 0,
               activation_fct = None,
               apply_softmax = False,
               convert_to_numpy: bool = True,
               convert_to_tensor: bool = False
               ):
        """
        Performs predicts with the CrossEncoder on the given sentence pairs.
        :param sentences: A list of sentence pairs [[Sent1, Sent2], [Sent3, Sent4]]
        :param batch_size: Batch size for encoding
        :param show_progress_bar: Output progress bar
        :param num_workers: Number of workers for tokenization
        :param activation_fct: Activation function applied on the logits output of the CrossEncoder. If None, nn.Sigmoid() will be used if num_labels=1, else nn.Identity
        :param convert_to_numpy: Convert the output to a numpy matrix.
        :param apply_softmax: If there are more than 2 dimensions and apply_softmax=True, applies softmax on the logits output
        :param convert_to_tensor:  Conver the output to a tensor.
        :return: Predictions for the passed sentence pairs
        """
        input_was_string = False
        if isinstance(sentences[0], str):  # Cast an individual sentence to a list with length 1
            sentences = [sentences]
            input_was_string = True

        inp_dataloader = DataLoader(sentences, batch_size=batch_size, collate_fn=self.smart_batching_collate_text_only, num_workers=num_workers, shuffle=False)

        if show_progress_bar is None:
            show_progress_bar = (logger.getEffectiveLevel() == logging.INFO or logger.getEffectiveLevel() == logging.DEBUG)

        iterator = inp_dataloader
        if show_progress_bar:
            iterator = tqdm(inp_dataloader, desc="Batches")

        if activation_fct is None:
            activation_fct = self.default_activation_function

        pred_scores = []
        self.model.eval()
        self.model.to(self._target_device)
        with torch.no_grad():
            for features in iterator:
                model_predictions = self.model(**features, return_dict=True)
                logits = activation_fct(model_predictions.logits)

                if apply_softmax and len(logits[0]) > 1:
                    logits = torch.nn.functional.softmax(logits, dim=1)
                pred_scores.extend(logits)

        if self.config.num_labels == 1:
            pred_scores = [score[0] for score in pred_scores]

        if convert_to_tensor:
            pred_scores = torch.stack(pred_scores)
        elif convert_to_numpy:
            pred_scores = np.asarray([score.cpu().detach().numpy() for score in pred_scores])

        if input_was_string:
            pred_scores = pred_scores[0]

        return pred_scores


    def _eval_during_training(self, evaluator, output_path, save_best_model, epoch, steps, callback):
        """Runs evaluation during the training"""
        if evaluator is not None:
            score = evaluator(self, output_path=output_path, epoch=epoch, steps=steps)
            if callback is not None:
                callback(score, epoch, steps)
            if score > self.best_score:
                self.best_score = score
                if save_best_model:
                    self.save(output_path)

    def save(self, path):
        """
        Saves all model and tokenizer to path
        """
        if path is None:
            return

        logger.info("Save model to {}".format(path))
        self.model.save_pretrained(path)
        self.tokenizer.save_pretrained(path)

    def save_pretrained(self, path):
        """
        Same function as save
        """
        return self.save(path)

## Evaluator Classes

In [ ]:
import itertools
from tqdm.notebook import trange
from torch.utils.data import DataLoader
import math
import torch
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers import InputExample
import logging
from datetime import datetime
import sys
import os
import gzip
from torch.nn import KLDivLoss
import csv

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_rmse(model, data, target):
    preds = model.predict(data)
    preds = [probability_to_ability(p) for p in preds]
    return mean_squared_error(target, preds, squared=False)

def construct_input_example_kl(row):
    p = get_probability(row.target_x, row.target_y)
    label = [p, 1-p]
    return InputExample(texts=[row.excerpt_x, row.excerpt_y], label=label)

def construct_input_example(row):
    p = get_probability(row.target_x, row.target_y)
    return InputExample(texts=[row.excerpt_x, row.excerpt_y], label=p)

def get_train_samples(train, n, kl, random_state=42):
    if kl:
        fn = construct_input_example_kl
    else:
        fn = construct_input_example
    return train.sample(n=n,random_state=random_state).reset_index().apply(fn, axis=1)

def get_test_samples(test_df):
    return test_df.apply(lambda row: [BASE_EXCERPT, row.excerpt], axis=1).values

def get_probability(t1, t2):
    return np.exp(t1) / (np.exp(t1) + np.exp(t2))

def probability_to_ability(p):
    return np.log((1-p) / p)

def gen_probability_matrix(df):
    m = np.zeros((len(df), len(df)))
    for i, j in itertools.permutations(range(len(df)), r=2):
        t1, t2 = df.iloc[i].target, df.iloc[j].target
        m[i][j] = get_probability(t1, t2)

    return m

def mle(pmat, max_iter=1000):
    n = pmat.shape[0]
    wins = np.sum(pmat, axis=0)
    params = np.ones(n, dtype=float)
    for _ in range(max_iter):
        tiled = np.tile(params, (n, 1))
        combined = 1.0 / (tiled + tiled.T)
        np.fill_diagonal(combined, 0)
        nxt = wins / np.sum(combined, axis=0)
        nxt = nxt / np.mean(nxt)
        if np.linalg.norm(nxt - params, ord=np.inf) < 1e-6:
            return nxt
        params = nxt
    print(f'faulty matrix: {pmat}')
    print(f'end params: {params}')
    raise RuntimeError('did not converge')


class SimpleBTEvaluator(SentenceEvaluator):
    def __init__(self, test_df):
        self.data = test_df.apply(lambda row: [row.excerpt, BASE_EXCERPT], axis=1).values
        self.target = test_df.target
        
    def __call__(self, model, output_path, epoch=-1, steps=-1):
        preds = model.predict(self.data, apply_softmax=True)
        preds = [probability_to_ability(p) for p in preds[:, 0]]
        error = mean_squared_error(self.target, preds, squared=False)
        print(f'mse for epoch {epoch} step {steps}: {error}')
        return error
    
class SampleBTEvaluator(SentenceEvaluator):
    def __init__(self, test_df, samples, kl):
        cross_df = cartesian(test_df, samples)
        self.samples = samples
        self.target = test_df.target
        self.data = cross_df.apply(lambda row: [row.excerpt_x, row.excerpt_y], axis=1).values
        self.rev_data = cross_df.apply(lambda row: [row.excerpt_y, row.excerpt_x], axis=1).values
        self.kl = kl
        
    def __call__(self, model, output_path, epoch=-1, steps=-1):
        preds = model.predict(self.data, apply_softmax=True)
        if self.kl:
            preds = preds[:, 0]
        preds = preds.reshape(preds.shape[0] // len(self.samples), len(self.samples))
        rev_preds = model.predict(self.rev_data, apply_softmax=True)
        if self.kl:
            rev_preds = rev_preds[:, 0]
        rev_preds = rev_preds.reshape(rev_preds.shape[0] // len(self.samples), len(self.samples))
        p_m = gen_probability_matrix(self.samples)
        
#         return preds, rev_preds
           
        ability_preds = []
                                       
        for i in trange(len(self.target)):
            inf_matrix = np.zeros((len(p_m) + 1, len(p_m) + 1))
            inf_matrix[:-1, :-1] = p_m
            inf_matrix[-1, :] = np.append(preds[i], 0)
            inf_matrix[:, -1] = np.append(rev_preds[i], 0)
            params = mle(inf_matrix)
            scaled_params = -np.log(params) - 1
            ability_preds.append(scaled_params[-1])
                                       
        error = mean_squared_error(self.target, ability_preds, squared=False)
        print(f'mse for epoch {epoch} step {steps}: {error}')
        
        csv_path = './val_metrics.csv'
        output_file_exists = os.path.isfile(csv_path)
        with open(csv_path, mode="a" if output_file_exists else 'w', encoding="utf-8") as f:
            writer = csv.writer(f)
            if not output_file_exists:
                writer.writerow(['epoch', 'steps', 'error'])
            writer.writerow([epoch, steps, error])
            
        return error
    
    
class DummyBTEvaluator(SentenceEvaluator):
    def __init__(self, test_df, samples):
        cross_df = cartesian(test_df, samples)
        self.samples = samples
        self.data = cross_df.apply(lambda row: [row.excerpt_x, row.excerpt_y], axis=1).values
        self.rev_data = cross_df.apply(lambda row: [row.excerpt_y, row.excerpt_x], axis=1).values
        
        target = cross_df.apply(lambda row: get_probability(row.target_x, row.target_y), axis=1).values
        target = torch.tensor(target)
        self.pred = target
        rev_target = cross_df.apply(lambda row: get_probability(row.target_y, row.target_x), axis=1).values
        rev_target = torch.tensor(rev_target)
        self.rev_pred = rev_target
        
        self.target = test_df.target
        
    def __call__(self, model, output_path, epoch=-1, steps=-1):
        preds = self.pred.reshape(self.pred.shape[0] // len(self.samples), len(self.samples))     
        rev_preds = self.rev_pred.reshape(self.rev_pred.shape[0] // len(self.samples), len(self.samples))
        p_m = gen_probability_matrix(self.samples)
           
        ability_preds = []
                                       
        for i in trange(len(preds)):
            inf_matrix = np.zeros((len(p_m) + 1, len(p_m) + 1))
            inf_matrix[:-1, :-1] = p_m
            inf_matrix[-1, :] = np.append(preds[i], 0)
            inf_matrix[:, -1] = np.append(rev_preds[i], 0)
            params = mle(inf_matrix)
            scaled_params = -np.log(params) - 1
            ability_preds.append(scaled_params[-1])
                                       
        error = mean_squared_error(self.target, ability_preds, squared=False)
        print(f'mse for epoch {epoch} step {steps}: {error}')
        return error
    
    
class SampleKLEvaluator(SentenceEvaluator):
    def __init__(self, test_df, samples):
        cross_df = cartesian(test_df, samples)
        self.samples = samples
        self.data = cross_df.apply(lambda row: [row.excerpt_x, row.excerpt_y], axis=1).values
        targets = cross_df.apply(lambda row: get_probability(row.target_x, row.target_y), axis=1).values
        self.rev_data = cross_df.apply(lambda row: [row.excerpt_y, row.excerpt_x], axis=1).values
        rev_targets = cross_df.apply(lambda row: get_probability(row.target_y, row.target_x), axis=1).values
        self.target = np.concatenate([targets, rev_targets])
        self.target = torch.tensor(self.target)
        self.target = torch.stack([self.target, 1 - self.target], axis=1)
        print(self.target.shape)
        
    def __call__(self, model, output_path, epoch=-1, steps=-1):
        preds = model.predict(self.data, apply_softmax=True, convert_to_tensor=True)
        preds = torch.log(preds)
        
        rev_preds = model.predict(self.rev_data, apply_softmax=True, convert_to_tensor=True)
        rev_preds = torch.log(rev_preds)
        
        loss = KLDivLoss(reduction='batchmean')(torch.cat([preds, rev_preds]), self.target.to(DEVICE))

        print(f'kl divergence for epoch {epoch} step {steps}: {loss}')
        return loss
    
class SampleEvaluator(SentenceEvaluator):
    def __init__(self, test_df, samples):
        cross_df = cartesian(test_df, samples) # it's important that the test_df is on the left. c(a, b) != c(b, a)
        self.cross_df = cross_df
        self.samples = samples
        self.data = cross_df.apply(lambda row: [row.excerpt_x, row.excerpt_y], axis=1).values
        self.rev_data = cross_df.apply(lambda row: [row.excerpt_y, row.excerpt_x], axis=1).values

        target = cross_df.apply(lambda row: get_probability(row.target_x, row.target_y), axis=1).values
        target = torch.tensor(target)
        self.target = torch.stack([target, 1 - target], axis=1)
        rev_target = cross_df.apply(lambda row: get_probability(row.target_y, row.target_x), axis=1).values
        rev_target = torch.tensor(rev_target)
        self.rev_target = torch.stack([rev_target, 1 - rev_target], axis=1)
        
    def __call__(self, model, output_path, epoch=-1, steps=-1):
        preds = model.predict(self.data, apply_softmax=True, convert_to_tensor=True)        
        rev_preds = model.predict(self.rev_data, apply_softmax=True, convert_to_tensor=True)
        
        return preds, rev_preds, self.target.to(DEVICE), self.rev_target.to(DEVICE)

In [ ]:
# unit test

# model = CrossEncoder('distilroberta-base', num_labels=2)
# x = SampleKLEvaluator(base_sample, df_test)
# x(model, '')

## Training

In [ ]:
import logging
import torch
# from sentence_transformers import CrossEncoder

KL = True

model_save_path = './fold'
if KL:
    model = CrossEncoder('distilroberta-base', num_labels=2, kl=True)
else:
    model = CrossEncoder('distilroberta-base', num_labels=1)
train_samples = get_train_samples(df_train, len(df_train), kl=KL, random_state=RANDOM_STATE)
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

# Configure the training
warmup_steps = math.ceil(len(train_dataloader) * 1 * 0.1) #10% of train data for warm-up
print("Warmup-steps: {}".format(warmup_steps))

model.fit(train_dataloader=train_dataloader,
          loss_fct=torch.nn.KLDivLoss(reduction='batchmean'),
          activation_fct = torch.nn.LogSoftmax(dim=1),
#               evaluator = SimpleBTEvaluator(df_test),
          evaluator = SampleBTEvaluator(df_test, base_sample, KL),
          evaluation_steps = 100,
          epochs=3,
          output_path=model_save_path) 

# CE
# model.fit(train_dataloader=train_dataloader,
#           epochs=3,
#           warmup_steps=warmup_steps,
#           output_path=model_save_path, 
#           evaluator = SampleBTEvaluator(df_test, base_sample, False),
#           evaluation_steps = 100,
#           use_amp=False)
model.save(model_save_path)


## Evaluation

First visually check one sample to make sure everything looks okay. Go through each step of the code one by one. Prove that it works / doesn't work one time.

In [ ]:
# load saved model

# model = CrossEncoder('/kaggle/input/crossencoder-kl1epoch/fold/')
# s = SampleEvaluator(df_test, base_sample)
# preds, rev_preds, target, rev_target = s(model, '')

In [ ]:
# preds[:, 0]

In [ ]:
# preds = preds[:, 0]
# rev_preds = rev_preds[:, 0]
# preds = preds.reshape(preds.shape[0] // len(base_sample), len(base_sample))  
# rev_preds = rev_preds.reshape(rev_preds.shape[0] // len(base_sample), len(base_sample))
# p_m = gen_probability_matrix(base_sample)

In [ ]:
# def get_mse(sample):
#     ability_preds = []
#     p_m = gen_probability_matrix(sample)

#     for i in trange(len(preds)):
#         inf_matrix = np.zeros((len(p_m) + 1, len(p_m) + 1))
#         inf_matrix[:-1, :-1] = p_m
#         inf_matrix[-1, :] = np.append(preds[i].cpu(), 0)
#         inf_matrix[:, -1] = np.append(rev_preds[i].cpu(), 0)
#         params = mle(inf_matrix)
#         scaled_params = -np.log(params) - 1
#         ability_preds.append(scaled_params[-1])

#     return mean_squared_error(df_test.target, ability_preds, squared=False)

# samples = []
# for i in trange(100):
#     sample = df_train.sample(n=10)
#     mse = get_mse(sample)
    

In [ ]:
# mean_squared_error(df_test.target, ability_preds, squared=False)

In [ ]:
# looks okay so far.

# preds[:10], target[:10]

In [ ]:
# reverse looks okay as well.

# rev_preds[:10], rev_target[:10]

In [ ]:
# for sure no errors code

# preds = []

# for i in trange(len(df_test)):
#     p_m_true = gen_probability_matrix(base_sample.append(df_test.iloc[0]))
#     params = mle(p_m_true)
#     scaled_params = -np.log(params) - 1
#     preds.append(scaled_params[-1])

In [ ]:
# mean_squared_error(df_test.target, preds, squared=False)

In [ ]:
# mean_squared_error(scaled_params, base_sample.append(df_test.iloc[0]).target, squared=False)

In [ ]:
# generate probability matrix with samples + add sample (using perfect targets)

# p_m = gen_probability_matrix(base_sample)

# inf_matrix = np.zeros((len(p_m) + 1, len(p_m) + 1))
# inf_matrix[:-1, :-1] = p_m
# inf_matrix[-1, :] = np.append(target.cpu().numpy()[:10][:, 0], 0)
# inf_matrix[:, -1] = np.append(rev_target.cpu().numpy()[:10][:, 0], 0)
# params = mle(inf_matrix)
# scaled_params = -np.log(params) - 1
# print(f'pred: {scaled_params[-1]}, true: {df_test.iloc[0].target}, rmse: {np.sqrt((scaled_params[-1] - df_test.iloc[0].target)**2)}')

In [ ]:
# generate probability matrix with samples + add sample (using actual predictions)

# p_m = gen_probability_matrix(base_sample)

# inf_matrix = np.zeros((len(p_m) + 1, len(p_m) + 1))
# inf_matrix[:-1, :-1] = p_m
# inf_matrix[:, -1] = np.append(preds.cpu().numpy()[:10][:, 0], 0)
# inf_matrix[-1, :] = np.append(rev_preds.cpu().numpy()[:10][:, 0], 0)
# params = mle(inf_matrix)
# scaled_params = -np.log(params) - 1
# print(f'pred: {scaled_params[-1]}, true: {df_test.iloc[0].target}, rmse: {np.sqrt((scaled_params[-1] - df_test.iloc[0].target)**2)}')

In [ ]:
# d = DummyBTEvaluator(df_test, base_sample)('', '')

In [ ]:
# BASE_EXCERPT = df[df.standard_error == 0].iloc[0].excerpt
# e = SimpleBTEvaluator(df_test)(model, '')

In [ ]:
# metrics = []

# for i in trange(50):
#     sample = raw.sample(n=10)
#     mse = SampleBTEvaluator(df_test, sample)(model, '')
#     metrics.append((mse, sample))

In [ ]:
# min([x[0] for x in metrics])

In [ ]:
# ability_preds = []
# p_m = gen_probability_matrix(base_sample)

# for i in trange(len(df_test)):
#     inf_matrix = np.zeros((len(p_m) + 1, len(p_m) + 1))
#     inf_matrix[:-1, :-1] = p_m
#     inf_matrix[-1, :] = np.append(preds[i], 0)
#     inf_matrix[:, -1] = np.append(rev_preds[i], 0)
#     params = mle(inf_matrix)
#     scaled_params = -np.log(params) - 1
#     ability_preds.append(scaled_params[-1])

# error = mean_squared_error(self.target, ability_preds, squared=False)
# print(f'mse for epoch {epoch} step {steps}: {error}')

In [ ]:
# preds = model.predict(test_samples)

In [ ]:
# x = torch.nn.functional.softmax(torch.tensor(preds), dim=1)[:, 0].numpy()

# mean_squared_error([probability_to_ability(p) for p in x],  df_test.target, squared=False)

In [ ]:
# df_test.target

In [ ]:
# BASE_EXCERPT = df[df.standard_error == 0].iloc[0].excerpt
# SimpleBTEvaluator

# test_samples = get_test_samples(df_test)
# rmse = get_rmse(model, test_samples, df_test.target)
# model.save(model_save_path)
# print(f'valid rmse for epoch {i}: {rmse}')

In [ ]:
# check to see if files were saved

# import os
# for dirname, _, filenames in os.walk('.'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

## Evaluation Using Bradley-Terry Optimization

In [ ]:
# import itertools
# from tqdm import tqdm_notebook
# from sklearn.metrics import mean_squared_error

# def gen_probability_matrix(df):
#     m = np.zeros((len(df), len(df)))
#     for i, j in itertools.permutations(range(len(df)), r=2):
#         t1, t2 = df.iloc[i].target, df.iloc[j].target
#         m[i][j] = get_probability(t1, t2)
#     return m


# def mle(pmat, max_iter=1000):
#     n = pmat.shape[0]
#     wins = np.sum(pmat, axis=0)
#     params = np.ones(n, dtype=float)
#     for _ in range(max_iter):
#         tiled = np.tile(params, (n, 1))
#         combined = 1.0 / (tiled + tiled.T)
#         np.fill_diagonal(combined, 0)
#         nxt = wins / np.sum(combined, axis=0)
#         nxt = nxt / np.mean(nxt)
#         if np.linalg.norm(nxt - params, ord=np.inf) < 1e-6:
#             return nxt
#         params = nxt
#     raise RuntimeError('did not converge')

# def evaluate(preds, rev_preds):
#     p_m = gen_probability_matrix(t_samples)
#     ability_preds = []

#     for i, row in tqdm_notebook(df_test.iterrows()):
#         inf_matrix = np.zeros((len(p_m) + 1, len(p_m) + 1))
#         inf_matrix[:-1, :-1] = p_m
#         inf_matrix[:, -1] = np.append(preds[i], 0)
#         inf_matrix[-1, :] = np.append(rev_preds[i], 0)
#         params = mle(inf_matrix)
#         scaled_params = -np.log(params) - 1
#         ability_preds.append(scaled_params[-1])
    
#     return mean_squared_error(df_test.target, ability_preds)

In [ ]:
# t_samples = df_train[df_train.excerpt.isin(raw_samples.excerpt_l.unique())].sample(n=1500, random_state=42).reset_index(drop=True)
# samples_with_base = pd.concat([t_samples, df[df.target == 0]])

In [ ]:
# mean_squared_error(samples_with_base.target, -np.log(mle(gen_probability_matrix(samples_with_base))) - 1, squared=False)

In [ ]:
# t_samples.target

## Training Code with Annotations

In [ ]:
# def smart_batching_collate(self, batch):
#     texts = [[] for _ in range(len(batch[0].texts))]
#     labels = []

#     for example in batch:
#         for idx, text in enumerate(example.texts):
#             texts[idx].append(text.strip())

#         labels.append(example.label)

#     tokenized = self.tokenizer(*texts, padding=True, truncation='longest_first', return_tensors="pt", max_length=self.max_length)
#     labels = torch.tensor(labels, dtype=torch.float if self.config.num_labels == 1 else torch.long).to(self._target_device)

#     for name in tokenized:
#         tokenized[name] = tokenized[name].to(self._target_device)

#     return tokenized, labels

# train_dataloader.collate_fn = smart_batching_collate 

# if use_amp:
#     from torch.cuda.amp import autocast
#     scaler = torch.cuda.amp.GradScaler()

# self.model.to(self._target_device)

# if output_path is not None:
#     os.makedirs(output_path, exist_ok=True)

# self.best_score = -9999999
# num_train_steps = int(len(train_dataloader) * epochs)

# # Prepare optimizers
# param_optimizer = list(self.model.named_parameters())

# no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
#     {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]

# optimizer = optimizer_class(optimizer_grouped_parameters, **optimizer_params)

# if isinstance(scheduler, str):
#     scheduler = SentenceTransformer._get_scheduler(optimizer, scheduler=scheduler, warmup_steps=warmup_steps, t_total=num_train_steps)

# if loss_fct is None:
#     loss_fct = nn.BCEWithLogitsLoss() if self.config.num_labels == 1 else nn.CrossEntropyLoss()


# skip_scheduler = False
# for epoch in trange(epochs, desc="Epoch"):
#     training_steps = 0
#     self.model.zero_grad()
#     self.model.train()

#     for features, labels in tqdm(train_dataloader, desc="Iteration", smoothing=0.05):
#         if use_amp:
#             with autocast():
#                 model_predictions = self.model(**features, return_dict=True)
#                 logits = activation_fct(model_predictions.logits)
#                 if self.config.num_labels == 1:
#                     logits = logits.view(-1)
#                 loss_value = loss_fct(logits, labels)

#             scale_before_step = scaler.get_scale()
#             scaler.scale(loss_value).backward()
#             scaler.unscale_(optimizer)
#             torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_grad_norm)
#             scaler.step(optimizer)
#             scaler.update()

#             skip_scheduler = scaler.get_scale() != scale_before_step
#         else:
#             model_predictions = self.model(**features, return_dict=True)
#             logits = activation_fct(model_predictions.logits)
#             if self.config.num_labels == 1:
#                 logits = logits.view(-1)
#             loss_value = loss_fct(logits, labels)
#             loss_value.backward()
#             torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_grad_norm)
#             optimizer.step()

#         optimizer.zero_grad()

#         if not skip_scheduler:
#             scheduler.step()

#         training_steps += 1

#         if evaluator is not None and evaluation_steps > 0 and training_steps % evaluation_steps == 0:
#             self._eval_during_training(evaluator, output_path, save_best_model, epoch, training_steps, callback)

#             self.model.zero_grad()
#             self.model.train()

#     if evaluator is not None:
#         self._eval_during_training(evaluator, output_path, save_best_model, epoch, -1, callback)

In [ ]:
##### Load model and eval on test set
# model = CrossEncoder(model_save_path)